# Individual Analysis for the Hanle Effect experiment

Use this template to carry out the analysis tasks for the experiment.  For reference, here are links to recommended Python resources: the [Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/) and the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) both by Jake VanderPlas.

We will be making use of both the [Uncertainties](https://pythonhosted.org/uncertainties/) and [LMFit](https://lmfit.github.io/lmfit-py/) packages in this notebook.

In [ ]:
import numpy as np  # import the numpy library functions.
import matplotlib.pyplot as plt # plotting functions
import uncertainties as unc # Uncertainties package.  Good for simple error propagation
import scipy.constants as const # Library of physical constants
import pandas as pd
%matplotlib inline

## Tasks for this analysis

#### Standard Anaysis

1. Use the peak maximum and minimum coil currents, the dwell time, and the sweep period to create a *calibration constant* that gives the change in coil current per MCS channel.

2. From the fits to the spectrum peaks, either with the Norland Interface application or LMFit, obtain the half-width at half-maximum (HWHM) in channel numbers. If you use LMFit, follow the tutorial in the **Examples** repository on fitting a complicated multi-peak function to a data set to fit the inverted peaks to two Lorentzians plus a constant background. 

3. Apply the calibration constant, and the conversion constant of current to magnetic field (14.3 gauss/amp) to obtain $\Delta B_{1/2}$ with uncertainty.

4. Calculate the lifetime of the <sup>3</sup>P<sub>1</sub> state of mercury.

#### Extended Analysis

The extended analysis involves converting the channels directly to magnetic field before running the peak fit. The fit itself then gives $\Delta B_{1/2}$ directly.

1. Use the peak maximum and minimum coil currents, the dwell time and sweep period to create a *calibration function* to convert channel number to coil current.  This function will be piecewise-linear.  Caution: you need to know the sign and magnitude of the current at the start of the scan (channnel zero).  This requires a careful observation of the ammeter vs. MCS sweep.

2. Apply your calibration function to the channel numbers to create an array of current values.  Plot the channel counts versus this array times the conversion constant of 14.3 gauss/amp, and see that the two peaks now lay exactly on top of each other.  If they do not do so, then there is a mistake.

3. Use LMFit to fit one inverted Lorentzian to the combined data set. From the fit result, obtain the HWHM which should be equal to $\Delta B_{1/2}$.  From this calculate the lifetime of the <sup>3</sup>P<sub>1</sub> state of mercury.

You may use either method of obtaining the final results. You do not need to carry out both.


### Standard Analysis: create a *calibration constant*

The calibration constant is the change in coil current per channel. Think carefully about the dwell time versus the function generator period.  How many channels would equal one period? (If you think it is 1024, you have misunderstood the problem.)  How many channels correspond to the time it takes to go from the maximum to the minimum current? Once you know this, you can calculate the constant. 

In [ ]:
# Conversion constant for current to magnetic field
conv_constant = 14.3 # gauss/amp

# Record the max and min currents here

I_max =  # amps
I_min =  # amps

# Record the dwell time and the sweep period

dwell_time = #seconds
sweep_period =  # seconds

# Calculate the change in current per channel

chan_per_period = 
cal_constant = 

# Calculate the change in magnetic field per channel
Delta_B_per_chan = 

print('Channels per period: {:.0f}'.format(chan_per_period))
print('Calibration constant: {:.4g} A/channel'.format(cal_constant))
print('Change in B-field per channel: {:.4g} gauss/channel'.format(Delta_B_per_chan))

## Use LMFit to analyze the vertical polarization data 

NOTE: If using the LabVIEW application data analysis result, skip this section and scroll down to **Calculate the lifetime of <sup>3</sup>P<sub>1</sub>** below.

The only datafile that needs to be analyzed numerically is the data with incoming polarization set to 0 degrees (vertical polarization) and the bucking coil on. Optionally, you may also analyze the same polarization with the bucking coil off, but this is only to compare to the other, better, dataset.

### Read in the data file

The raw data files are tab separated two-column files with a one-line header that does not matter for multichannel scaling files.  You need to skip the header and name the columns to create a dataframe.

Use the **Pandas** `read_csv()` function to read the raw data file from the Norland MCA as follows:

    Hanle_Vpol = pd.read_csv('Hanle_vertical_polarization_data.txt',header=0,names=['Chan','Counts'],sep='\t')
    
The dataframe will be named `Hanle_Vpol`  with columns `Hanle_Vpol['Chan']` ad `Hanle_Vpol['Counts']`. 

In [ ]:
# Read the data in as described.  
# You code this

# With the Python display() function, you can print a nice table


#### Plot it
Make a plot to check that it looks right.

In [ ]:
# Sanity plot example

myfig = plt.figure(figsize=(10,7))
plt.grid()
plt.xlabel('Channel')
plt.ylabel('Counts')
plt.title('Hanle data for vertical polarization')
plt.plot(Hanle_data['Chan'],Hanle_data['Counts'],'.');

### Fit the data to a model

Now you should be ready to apply the procedure described in the **Composite Model Demo** notebook.

Below is a reminder of the basic steps.

#### Make the model

For *Standard Analysis,* the model fit will consist of two Lorentzian peaks and a Constant background. 

For *Extended Analysis,* the model fit will have only one Lorentzian and a Constant background.  The process is the same for either approach, except you do not need to use the "prefix" specifier when there is only one peak (and only one set of peak parameters).

The Lorentzian lineshape has the form
$$f(x;A,\mu,\sigma) = \frac{A}{\pi}\left[\frac{\sigma}{(x-\mu)^2 + \sigma^2}\right]\;.$$ 

Note that the height of the peak at the center ($x=\mu$) is equal to $\frac{A}{\pi\sigma}$ and that the full-width at half-maximum is $2\sigma$.

In [5]:
# Import a gaussian peak and second order polynomial for background

from lmfit.models import ConstantModel, LorentzianModel

# create an instance of the model
# Note use of prefixes to keep parameters separate
model1 = ConstantModel() + LorentzianModel(prefix='p1_') + LorentzianModel(prefix='p2_')

params1 = model1.make_params()

display(params1)

name,value,initial value,min,max,vary,expression
c,0.00000000,None,-inf,inf,True,
p1_amplitude,1.00000000,None,-inf,inf,True,
p1_center,0.00000000,None,-inf,inf,True,
p1_sigma,1.00000000,None,0.00000000,inf,True,
p2_amplitude,1.00000000,None,-inf,inf,True,
p2_center,0.00000000,None,-inf,inf,True,
p2_sigma,1.00000000,None,0.00000000,inf,True,
p1_fwhm,2.00000000,None,-inf,inf,False,2.0000000*p1_sigma
p1_height,0.31830990,None,-inf,inf,False,"0.3183099*p1_amplitude/max(1e-15, p1_sigma)"
p2_fwhm,2.00000000,None,-inf,inf,False,2.0000000*p2_sigma


#### Set the fit parameter starting points

The first peak is shown as an example.  To set the amplitude, note that the amplitude $A$ in terms of the peak height $h$ is given by

$$A=h\sigma\pi$$

You can estimate decent starting values by simply reading the graph to obtain the full-width at hald maximum, the peak locations, the peak heights and the lelev of the baseline (background).

In [ ]:
# Example of how to set the parameters. Setting vary=False makes that parameter fixed
params1['p1_center'].set(value=275, vary=True)
params1['p1_sigma'].set(value=30, vary=True)
params1['p1_amplitude'].set(value=-6000.0*30.0*np.pi, vary=True)

# You do the rest

display(params1)

#### Fit the model

The code below is an example.  Note the use of wieghts from Poisson statistics of counting and also the commands to resize the plot from the fitting routine.

In [ ]:
# fit the model
model1_fit = model1.fit(Hanle_data['Counts'], params1, x=Hanle_data['Chan'], weights=1/np.sqrt(Hanle_data['Counts']))

print(model1_fit.fit_report(show_correl=False))

myfig=plt.figure(figsize=(10,7))
model1_fit.plot('.',fig=myfig)
plt.grid()
plt.xlabel(r'Channel')
plt.ylabel(r'Counts')
plt.title('Hanle data for vertical polarization');

### Save the peak half-widths

When you have obtained the best values, save the peak HWHM parameters.  I like to make a dataframe.  

In [ ]:
# Helper function

def get_uparm(params, name):
    '''
    Helper function to extract parameter value and uncertainty into
    a ufloat object.
    '''
    return unc.ufloat(params[name].value, params[name].stderr)

In [ ]:
# Extract the "sigma" parameters and sve them as uncertainty objects

# You code this


In [ ]:
# Here is an example of how to make a datframe.  You can add other columns using the method shown.
Hanle_peaks_df = pd.DataFrame()
Hanle_peaks_df['Number'] = [1,2]
Hanle_peaks_df['HWHM (ch)'] = sigmas # An array of the sigma parameters

display(Hanle_peaks_df)

## Calculate the lifetime of <sup>3</sup>P<sub>1</sub>

Now, use the HWHM in units of magnetic field to obtain the lifetime.

**NOTE:** If you are importing the HWHM from the LabVIEW data analysis, here is where you use it.

### Find $\Delta B_{1/2}$ from half-width

Apply your calibration constant and the current-to-magnetic-filed conversion connstant to obtain $\Delta B_{1/2}$.


In [ ]:
# Add columns to the dataframe
# You code this

display(Hanle_peaks_df)

In [ ]:
# If using LabVIEW result, make an uncertainty object with the half-width and its uncertainty, 
# and then multiply by the conversion and calibration constants.



### g-factor calulation

You need the g-factor for the $^3$P$_1$ state of mercury.  From the <i>Zeeman Effect in Mercury</i> instructions, the Lande g-factor formula in the L-S coupling scheme is given by

$$ g_J = 1 + \frac{J(J+1) + S(S+1) - L(L+1)}{2J(J+1)} $$

You can either calculate this directly, or make a function (that you might use elsewhere).

In [ ]:
# Function for calculating the Lande g-factor for J=L+S

def g_J(S=1, L=1, J=1):
    '''Calculate the Lande g-factor for Russell-Saunders coupling.
    '''
    # You code this, or just calculate directly
    return g


# Test of functon
print('g-factor for 1S_1/2 =',g_J(0.5,0,0.5))
print('g-factor for 3P_1 =',g_J(1,1,1))

### Lifetime calculation

Equation (4) in the instructions gives the lifetime of the excited state $\tau$, as expected from the Hanle effect linewidth:

$$ \frac{1}{\tau} = 2g_J\frac{\mu_B}{\hbar}\left|\Delta B_{1/2}\right| $$

where $\Delta B_{1/2}$ is the half-width at half-maximum of the "absorption peak", that is, the inverted Lorentzian peak in the scan of the emission vs. magnetic field with vertical incident polarization.

You can obtain the Bohr magneton $\mu_B$ from the SciPy constants database: `const.value('Bohr magneton')` and `const.hbar` gives $\hbar$.


In [ ]:
# Calculalate the lifetime

# First, I make a function
def tau(Del_B, g=1.5):
    '''Calculate the excited state lifetime in nanoseconds.
       Del_B is in gauss
    '''
    # You code this
    
    return tau_ns                        # Return tau in nanoseconds

In [ ]:
# Add another column with lifetimes

# take the average of the two to give the best value

tau_best =
print('Lifetime of the 3P_1 state of Hg, by the Hanle effect: {:.uP} ns'.format(tau_best))

## Extended Analysis

Below are the steps to complete the extended analysis by converting the channel numbers to current and then B-field before running a single peak fit.

### Extended Analysis: create a *calibration function*

Think carefully about the current versus time graph: What is the value of the current at channel 0? What is the value of the current at 1/2 of the function generator period? At which channel would this occur? Because the function generator makes a triangle wave, the shape of the function is piecewise-linear.

There are a couple of ways to make a piece-wise function.  A general way to do this is to use a "Heaviside" function $H(x)$ (named after Oliver Heaviside, a 19th century English scientist), which is a function that is equal to 1 for $x>0$ and 0 for $x\le0$.  Then do tricks with the channel numbers to convert them to positive or negative numbers.  By multiplying the function you want (e.g., a line) by the Heaviside function, it will appear when the channel numbers make the Heaviside function 1 and block it when the channel numbers make it 0.  

For example, if the function is a line with a negative slope for channels between 0 and $N$ and a line with a positive slope for channels above $N$, and these lines can be specified by their slopes $m_1$ and $m_2$, and intercepts $b_1$ and $b_2$, one could make a piecewise line $f(x)$ with

$$ f(x) = H(N-x)[m_1x + b_1] + H(x-N)[m_2x + b_2] $$

In this case, the two slopes are the same magntitude but reversed in sign: $m_2 = -m_1$.  To make the lines join, one sets $b_1$ at channel 0, and $b_2$ found from the intersection of the line going down and the line going up which must cross at 1/2 the period.

The heaviside function is available from NumPy as `np.heaviside()`.  You should look it up for details on how to use it.

A second method would be to use the SciPy function `scipy.signal.sawtooth`, which makes a triangle waveform when the `width` parameter is set to 0.50.  The sawtooth starts at -1 and goes to +1 with a period of $2\pi$.  You would need to rescale and offset the values produced by this function so that it ranges between the maximum and minimum current measured, and also scale the  argument appropriately, since the function is periodic whenever the argument $t = 2\pi$.  For example, if the maximum and minimum currents are $I_{max}$ and $I_{min}$, the peak-to-peak change in current is $I_{max}-I_{min}$ and the offset would be the average of these currents. Then convert channels to successive time values, in seconds. The frequency is $1/T$, where $T$ is the period, so the time values $t$ are fed into the sawtooth function as $2\pi t/T$. 

In [ ]:
# Make an array that matches the channel numbers
chan = np.asarray(Hanle_data['Chan'])
print('channel array:', chan)

# You code the rest Make a calibration function according to 
# one of the schemes described

def cal_fn(x, dwell=dwell_time, T=sweep_period, I_max=I_max, I_min=I_min):
    '''Calibration function to convert channel number to current'''

    # You code this
    
    return I


In [ ]:
# Test plot to make sure the calibration function looks right
plt.xlabel('Channel')
plt.ylabel('Current (A)')
plt.title('Calibration test')
plt.plot(chan, cal_fn(chan))
plt.grid(True);

### Apply the calibration function

Plot the channel counts against the calibrated current.  The two peaks should collapse to one, with the current nearly symmetrical about the center of the peak.

In [ ]:
# Sanity plot
# You code this


### Fit the modified data

Build a model with one Lorentzian peak and a constant background.  You will not need to worry about the peak "prefix".

In [ ]:
# create an instance of the model
# Note use of prefixes to keep parameters separate
model2 = ConstantModel() + LorentzianModel()

# You code the rest of the setup

In [ ]:
# Set the starting values of the fit parameters, based on the graph.


In [ ]:
# Carry out the fit and plot the results


### Calculate the lifetime of <sup>3</sup>P<sub>1</sub> state

In [ ]:
# Obtain the sigma parameter
sigma = get_uparm(model2_fit.params,'sigma')
print('Delta-B_1/2 (G) =', sigma)

# Calculate the lifetime of the 3P_1 state
tau = tau(sigma)
print('Lifetime of the 3P_1 state of Hg, by the Hanle effect, 2nd method: {:.uP} ns'.format(tau))